In [72]:
import numpy as np
import pandas as pd

In [73]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [74]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA3b/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [75]:
mineralogy = mineralogy.dropna()

In [76]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [77]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [78]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [79]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
70704,61.892754,0.567829,16.886432,1.319497,3.844936,0.110648,3.275339,5.607538,2.843240,3.651787
70705,61.889843,0.567864,16.886735,1.319692,3.845526,0.110682,3.276179,5.608836,2.843191,3.651453
70706,61.887097,0.567896,16.887020,1.319876,3.846083,0.110715,3.276972,5.610060,2.843144,3.651138
70707,61.884539,0.567927,16.887285,1.320048,3.846601,0.110745,3.277711,5.611200,2.843100,3.650844
70708,61.882192,0.567955,16.887529,1.320206,3.847077,0.110773,3.278388,5.612246,2.843060,3.650575


In [80]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
70704,61.892754,0.567829,16.886432,1.319497,3.844936,0.110648,3.275339,5.607538,2.843240,3.651787
70705,61.889843,0.567864,16.886735,1.319692,3.845526,0.110682,3.276179,5.608836,2.843191,3.651453
70706,61.887097,0.567896,16.887020,1.319876,3.846083,0.110715,3.276972,5.610060,2.843144,3.651138
70707,61.884539,0.567927,16.887285,1.320048,3.846601,0.110745,3.277711,5.611200,2.843100,3.650844
70708,61.882192,0.567955,16.887529,1.320206,3.847077,0.110773,3.278388,5.612246,2.843060,3.650575
...,...,...,...,...,...,...,...,...,...,...
234326,63.811892,0.543454,16.647964,1.189257,3.452511,0.089058,2.739272,4.772073,2.872267,3.882253
234327,63.811248,0.543462,16.648058,1.189301,3.452644,0.089065,2.739445,4.772347,2.872259,3.882172
234328,63.810589,0.543471,16.648154,1.189346,3.452780,0.089072,2.739623,4.772627,2.872250,3.882089
234329,63.809913,0.543480,16.648252,1.189392,3.452919,0.089079,2.739805,4.772913,2.872242,3.882004


In [81]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]
mineralogy3 = mineralogy.iloc[90000:135000]

In [82]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA3b/area3b_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA3b/area3b_2.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA3b/area3b_3.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [83]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA3b/area3b_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA3b/area3b_2_results_excel.txt")
df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA3b/area3b_3_results_excel.txt")

In [84]:
df_final = pd.concat([df_final1, df_final2, df_final3])

In [85]:
df_final

,Q,P,K
SAMPLE,,,
70704,13.235055,44.179463,23.984598
70705,13.230227,44.181123,23.982549
70706,13.225672,44.182674,23.980674
70707,13.221398,44.184170,23.978868
70708,13.217483,44.185501,23.977238
...,...,...,...
234326,16.392233,42.938939,25.370981
234327,16.391178,42.939413,25.370499
234328,16.390125,42.939862,25.370020


## QAPF classification

In [86]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [87]:
df_final["QAPF"].unique()

array(['quartz monzonite', 'quartz monzodiorite\nquartz monzogabbro',
       'monzo granite'], dtype=object)

In [88]:
df_final.to_excel("../_CIPW/CIPW/AREA3b/qapf.xlsx", index=True)

-------

In [89]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA3b/QAPF_counts.xlsx")

-----

In [90]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area3_b.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [91]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
2187,671151.209555,7.734892e+06,55.0,W,9.0,26.342266,32.302614,34.298288,monzo granite
3800,671687.565203,7.727472e+06,55.0,W,9.0,23.234255,39.186848,24.550674,monzo granite
3819,671415.394591,7.731167e+06,55.0,W,9.0,20.238474,43.591265,20.630025,granodiorite
4080,670508.338667,7.734853e+06,55.0,W,9.0,16.409475,43.528818,24.750370,quartz monzonite
4088,680683.051474,7.737473e+06,55.0,W,9.0,16.813600,38.598839,30.498563,quartz monzonite
4189,672065.958431,7.731232e+06,55.0,W,9.0,17.855997,40.693400,24.974282,monzo granite
4334,667785.476262,7.736481e+06,55.0,W,9.0,12.959965,44.129657,23.809950,quartz monzonite
4426,676998.398496,7.725990e+06,55.0,W,9.0,10.653355,40.967293,29.251499,quartz monzonite
4473,680092.785619,7.728082e+06,55.0,W,9.0,9.635997,44.692886,22.659857,quartz monzodiorite\nquartz monzogabbro
4512,676214.749308,7.727801e+06,55.0,W,9.0,7.866771,46.614313,21.609282,quartz monzodiorite\nquartz monzogabbro


In [92]:
QAPF_control.to_excel("../_CIPW/CIPW/AREA3b/QAPF_control_points.xlsx", index=True)